# Homework 2
### Pierre-Antoine Desplaces, Anaïs Ladoy, Lou Richard

In [ ]:
# Import libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import json

### Question 1

Obtain the 200 top-ranking universities in www.topuniversities.com ([ranking 2018](https://www.topuniversities.com/university-rankings/world-university-rankings/2018)). In particular, extract the following fields for each university: name, rank, country and region, number of faculty members (international and total) and number of students (international and total). Some information is not available in the main list and you have to find them in the [details page](https://www.topuniversities.com/universities/ecole-polytechnique-fédérale-de-lausanne-epfl).
Store the resulting dataset in a pandas DataFrame and answer the following questions:
- Which are the best universities in term of: (a) ratio between faculty members and students, (b) ratio of international students?
- Answer the previous question aggregating the data by (c) country and (d) region.

Plot your data using bar charts and describe briefly what you observed.


In [ ]:
URL1 = 'https://www.topuniversities.com/university-rankings/world-university-rankings/2018'
URL2 = 'http://timeshighereducation.com/world-university-rankings/2018/world-ranking'

In [ ]:
# Do the request
r = requests.get(URL1)

Inspecting the DOM of the website, we find that the ranking datas are stored in a text file at https://www.topuniversities.com/sites/default/files/qs-rankings-data/357051.txt as we can see here :

In [ ]:
# 357051.txt
id1 = r.text.find("357051.txt")
r.text[id1-88:id1+11]

We thus request the datas from this URL, keeping only the 200 first elements.

In [ ]:
data_QS_URL = 'https://www.topuniversities.com/sites/default/files/qs-rankings-data/357051.txt'
data_QS = requests.get(data_QS_URL)
#create the dataframe
df_QS = pd.DataFrame(data_QS.json()['data'][:200])

We now keep the relevant columns.

In [ ]:
# select only the useful columns
df_ranking = pd.DataFrame({"University" : df_QS.title,\
                           "Rank" : df_QS.rank_display,\
                           "Score" : df_QS.score,\
                           "Country": df_QS.country,\
                           "Region": df_QS.region, "URL" : df_QS.url},\
                          columns = ['Rank', 'University', 'Score', 'Country', 'Region', 'URL'])

In [ ]:
df_ranking

We observe that the 200th university has a rank = 201, this is because there is no 198th (we can see that the last 195th should be the 197th, and the next rank is 199).

We now have to get the number of faculties and students for each university. For this, we have to go the page of the university on the topuniversities website, and collect these information. Inspecting these webapges, we find that the number of faculty members is in the div number of the div total faculty, the number of international faculty members is in the div number of the div inter faculty, the number of students is in the div number of the div total students, and finally the number of international students is in the div number of the div total inter.

In [ ]:
# fetch the university informations from its webpage
def get_numbers_of(url):
    # get the data from the university page
    r = requests.get('https://www.topuniversities.com'+url)
    soup = BeautifulSoup(r.text, "html.parser")
    # find the corresponding tag
    try:
        staff = soup.find('div', class_="total faculty").find('div', class_="number").string
        staff = int(str(staff).replace('\n', "").replace(",", ""))
    except:
        staff = np.nan
    try :
        inter_staff = soup.find('div', class_="inter faculty").find('div', class_="number").string
        inter_staff = int(str(inter_staff).replace('\n', "").replace(",", ""))
    except:
        inter_staff = np.nan
    try : 
        students = soup.find('div', class_="total student").find('div', class_="number").string
        students = int(str(students).replace('\n', "").replace(",", ""))
    except:
        students = np.nan
    try : 
        inter_students = soup.find('div', class_="total inter").find('div', class_="number").string
        inter_students = int(str(inter_students).replace('\n', "").replace(",", ""))
    except:
        inter_students = np.nan
    
    return staff, inter_staff, students, inter_students

We fetch the results into new columns in our dataframe.

In [ ]:
df_ranking['Faculty Members'], \
df_ranking['International Faculty Members'], \
df_ranking['Total Students'], \
df_ranking['International Students'] = zip(*df_ranking['URL'].map(get_numbers_of))

We can now remove the URL columns, as it is not useful anymore.

In [ ]:
df_ranking = df_ranking.drop('URL', axis=1)
df_ranking

#### a) Which are the best universities in term of ratio between faculty members and students ?

In [ ]:
df_ranking['% Fac Members/Students'] = df_ranking['Faculty Members']/df_ranking['Total Students']*100

In [ ]:
df_ranking.sort_values('% Fac Members/Students', ascending=False)

In [ ]:
bars("Ratio Faculty Members and Students per Country", df_ranking.sort_values('% Fac Members/Students', ascending=False)[:20], 'University', '% Fac Members/Students')

We observe that this ranking is different from the original ranking, but the 10 first university are almost all in the top 20 of the initial ranking.

#### b) Which are the best universities in term of ratio of international students?

In [ ]:
df_ranking['%International Students'] = df_ranking['International Students']/df_ranking['Total Students']*100

In [ ]:
df_ranking.sort_values('%International Students', ascending=False)

In [ ]:
bars("Ratio Faculty Members and Students per Country", df_ranking.sort_values('%International Students', ascending=False)[:20], 'University', '%International Students')

Here, the results are completely different from the original ranking.

#### c) Aggregating the data by country

In [ ]:
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt

def bars(t, data, xlab, ylab):
    sns.set_style('darkgrid')
    fig, ax = plt.subplots(figsize = (15,8))
    ax.set_title(t, fontsize=15, fontweight='bold')
    sns.barplot(x=xlab, y=ylab, data=data, saturation=0.7, errcolor='.7')
    plt.xticks(rotation=90)
    plt.show()

In [ ]:
bars("Ratio Faculty Members and Students per Country", df_ranking.sort_values('Country'), 'Country', '% Fac Members/Students')

In [ ]:
bars("International Students per Country", df_ranking.sort_values('Country'), 'Country', '%International Students')

#### d) Aggregating the data by region

In [ ]:
bars("Ratio Faculty Members and Students per Region", df_ranking.sort_values('Region'), 'Region', '% Fac Members/Students')

In [ ]:
bars("International Students per Region", df_ranking.sort_values('Region'), 'Region', '%International Students')

### Question 2

Obtain the 200 top-ranking universities in www.timeshighereducation.com ([ranking 2018](http://timeshighereducation.com/world-university-rankings/2018/world-ranking)). Repeat the analysis of the previous point and discuss briefly what you observed.

Using Postman, a API request was captured on the ranking, containing a json file with all the data.

In [ ]:
times_r=requests.get('https://www.timeshighereducation.com/sites/default/files/the_data_rankings/world_university_rankings_2018_limit0_369a9045a203e176392b9fb8f8c1cb2a.json')
data=times_r.text

We decode the json file by keeping only the first 200 data items (each data item corresponds to an university, sorting according its ranking). Furthemore, we just extract the useful columns (name, location, rank, total number of students, percentage of international students and student/staff ratio). 

As we are asked to sort the universities according their ratio between faculty members and students, we're already converted the student/staff ratio to staff/student ratio and convert the ratio in percentage. 

In [ ]:
df_times = pd.DataFrame(json.loads(data)['data'][:200],columns=['name','location','rank','stats_number_students','stats_pc_intl_students','stats_student_staff_ratio'])
df_times['stats_pc_intl_students'] = df_times['stats_pc_intl_students'].astype(str).str.replace('%','').astype(int)
df_times['stats_number_students'] = df_times['stats_number_students'].astype(str).str.replace(',','').astype(float)
df_times['stats_student_staff_ratio'] = 100/(df_times['stats_student_staff_ratio'].astype(float))
df_times.columns = ['University','Country','Rank','Total Students','% International Students','% Fac Members/Students']
df_times


We don't have the region information in the second website so we decide to use the information collected in the first one.

In [ ]:
conv_to_cont = pd.DataFrame(data_QS.json()['data'],columns=['country','region']).drop_duplicates()
conv_to_cont.columns=['Country','Region'];
conv_to_cont.head()

Then, we merged the two dataframes on the country attribute.

In [ ]:
df_times_final=pd.merge(df_times, conv_to_cont,how='left',left_on="Country",right_on="Country")


In [ ]:
df_times_final[df_times_final.Region.isnull()]

As we can see below, two universities have no defined Region because the information about these countries was missing in the first ranking. We complete manually the missing information as just two universities are concerned and they won't appear on the barplot when we'll aggregate ranking by region otherwise.

In [ ]:
df_times_final.at[178,'Region']='Europe'
df_times_final.at[193,'Region']='Europe'

Some informations are not explicitely present in our data as the number of international students and the total number of faculty members but we can infer them with the other columns according the following formulas :

Number of international students : *0.01 x Percentage of International Students x Total number of Students*

Total number of faculty members : *Total number of students x 0.01 x Ration between Faculty members and Students*

Nethertheless, we cannot infer the number of international faculty members but it is not essential for the results.

In [ ]:
df_times_final['International Students'] = df_times_final.apply(lambda row:row["Total Students"] * 0.01 * row["% International Students"], axis=1).round()
df_times_final['Faculty members'] = df_times_final.apply(lambda row: row["Total Students"] * 0.01 * row["% Fac Members/Students"], axis=1).round()
df_times_final

#### a) Which are the best universities in term of ratio between faculty members and students ?

In [ ]:
df_times_final.sort_values('% Fac Members/Students', ascending=False)

In [ ]:
bars("Ratio Faculty Members and Students per Country", df_times_final.sort_values('% Fac Members/Students', ascending=False)[:20], 'University', '% Fac Members/Students')

#### b) Which are the best universities in term of ratio of international students?

In [ ]:
df_times_final.sort_values('% International Students', ascending=False)

In [ ]:
bars("International Students per Country", df_times_final.sort_values('% International Students', ascending=False)[:20], 'University', '% International Students')

#### c) Aggregating the data by country

In [ ]:
bars("Ratio Faculty Members and Students per Country", df_times_final.sort_values('Country'), 'Country', '% Fac Members/Students')

In [ ]:
bars("International Students per Country", df_times_final.sort_values('Country'), 'Country', '% International Students')

#### d) Aggregating the data by region

In [ ]:
bars("Ratio Faculty Members and Students per Region", df_times_final.sort_values('Region'), 'Region', '% Fac Members/Students')


In [ ]:
bars("International Students per Region", df_times_final.sort_values('Region'), 'Region', '% International Students')

### Question 3


Merge the two DataFrames created in questions 1 and 2 using university names. Match universities' names as well as you can, and explain your strategy. Keep track of the original position in both rankings.

### Question 4
Find useful insights in the data by performing an exploratory analysis. Can you find a strong correlation between any pair of variables in the dataset you just created? Example: when a university is strong in its international dimension, can you observe a consistency both for students and faculty members?

### Question 5

Can you find the best university taking in consideration both rankings? Explain your approach.